In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2" 


import torch_geometric
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, GraphUNet
from torch_geometric.data import Batch
from torch_geometric.utils import to_dense_adj
from tqdm import tqdm


from utils.data import GraphDataModule, save_prediction
from utils.training import train_model
from utils.metrics import evaluate_model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
data_module = GraphDataModule("./data", num_workers=1)
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

Time taken to load ./data\hr_train.csv: 3.77698016166687 seconds
Time taken to load ./data\lr_train.csv: 0.6752758026123047 seconds
Time taken to load ./data\lr_test.csv: 0.39960789680480957 seconds


Converting vectors to graphs: 100%|██████████| 112/112 [00:00<00:00, 1171.93it/s]


In [ ]:
class UpscalerGNN(nn.Module):
    def __init__(self, hidden_nodes, n_layers):
        super().__init__()
        self.input_nodes = 160
        self.output_nodes = 268
        
        self.layer1 = nn.Linear(self.input_nodes, hidden_nodes)
        self.gcn1 = GCNConv(hidden_nodes, hidden_nodes)
        
        self.layer2 = nn.Linear(self.output_nodes-self.input_nodes, hidden_nodes)
        


    @property
    def device(self):
        return next(self.parameters()).device

    def forward(self, samples: Batch):
        batch_size = samples.batch_size
        X1 = self.layer1(samples.x)
        X2 = self.gcn1(X1, )
                
        X2 = torch.eye(self.output_nodes-self.input_nodes).to(self.device)
        X2 = self.layer2(X2)
        
        
        
        

        # Initialize output matrix batch
        matrix = torch.zeros((batch_size, self.num_nodes_output, self.num_nodes_output), device=x.device)

        # Create mask for upper triangular part
        mask = torch.triu(torch.ones(self.num_nodes_output, self.num_nodes_output)).bool().to(x.device)

        # Fill upper triangular part
        matrix[:, mask] = x

        # Mirror upper triangular to lower triangular (make symmetric)
        matrix = matrix + matrix.transpose(1, 2) - torch.diag_embed(torch.diagonal(matrix, dim1=1, dim2=2))

        return matrix


In [4]:
batch,target_batch = next(iter(train_loader))
input_dim = batch[0].x.shape[0]
output_dim = target_batch[0].x.shape[0]
print(input_dim,output_dim)

160 268


In [ ]:
model = SuperResMLP(input_dim, output_dim, num_hidden_nodes=(input_dim+output_dim)//2, n_layers=1).to(device)

In [ ]:
criterion = nn.MSELoss()

train_loss_history, val_loss_history, lr_history, best_model_state_dict = train_model(
    model=model, 
    train_dataloader=train_loader, 
    val_dataloader=val_loader,
    criterion=criterion,
    num_epochs=100,
)

In [ ]:
model.load_state_dict(best_model_state_dict)
loss = evaluate_model(model, val_loader)
print(loss)

torch.save(model, 'model.pth')

In [ ]:
model = torch.load("model.pth", weights_only=False)

In [ ]:
test_dataloader = data_module.test_dataloader()

In [ ]:
submission_file = "outputs/test/submission.csv"
save_prediction(model, test_dataloader, submission_file)

In [ ]:
df = pd.read_csv(submission_file)

In [ ]:
!kaggle competitions submit -c dgl-2025-brain-graph-super-resolution-challenge -f outputs/test/submission.csv -m "test"
